Surface Fuel Updates.. buckle up

There will be 4 GridFire runs representing Year 5, Year 10, Year 15, and Year 20.

We have 4 year intervals in our ee_treatment simulation images, the pixel values and their Year Interval correspond:

1 (years 1-5) 
2 (years 6-10) 
3 (years 11-15)
4 (years 16-20)

We want to generate newly updated FM40 layers for each GridFire run year. 
we will be updating only those treated pixels with value 1, 2, 3, and 4 from our ee_treatment() images.

For each of those pixel values (1,2,3,4) we will create a new updated FM40 image each from the original and re-treatment images. 

The workflow is this:

For original treatment and re-treatment (retreatment_only) images:
* mask to only pixels with that given value, 
* use a simplified lookup table .csv on Cloud Storage (containing from_codes and to_codes) to remap the baseline FM40 values to their updated values in those treated pixels
* then fill in the 'untreated pixels' with the baseline FM40 image

Finally, mosaic both remapped FM40s together, taking the lowest FM40 at each pixel. 

Expected Output is three Updated FM40 images per RunID

Example: RUNID1 = SN WUI 500k
Outputs:
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_pctGroup25 (updated in ee_treatment pixels==1)
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_pctGroup50 (updated in ee_treatment pixels==2)
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_pctGroup75 (updated in ee_treatment pixels==3)
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_pctGroup100 (updated in ee_treatment pixels==4)

We also are exporting them to Google Drive in case there is a need
* Google Drive - RunID1_FM40_pctGroup25.tif
* Google Drive - RunID1_FM40_pctGroup50.tif
* Google Drive - RunID1_FM40_pctGroup75.tif
* Google Drive - RunID1_FM40_pctGroup100.tif


In [15]:
import ee
import os
import math
from datetime import datetime
from src.utils import ee_treatments
from src.utils import runs
from src.utils.check_exists import check_exists
from src.utils.yml_params import get_export_params
from src.utils.ee_csv_parser import parse_txt, to_numeric
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='mas-gee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'fm40_updates'
ee.data.setWorkloadTag(tag)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INPUT YOUR TREATED IMAGE PATH BELOW, THEN RUN ALL

In [16]:
# Plug in treatment scenario ee.Image path
ee_treated_path = "projects/mas-gee/assets/treatment_scenarios/RunID27/SC_Fire_2m_20231020"
retreatment_path = "projects/mas-gee/assets/treatment_scenarios/RunID27/SC_Fire_2m_20231103_retreatment_only"

In [17]:
ee_treated = ee.Image(ee_treated_path)
retreated = ee.Image(retreatment_path)
baseline_fm40 = ee.Image("projects/pyregence-ee/assets/conus/landfire/fbfm40/LF2022_FBFM40_220")

In [18]:
# test ee blob table encoding working properly
blob = ee.Blob("gs://landfire/LFTFCT_tables/million-acres-LUTs/LANDFIRE fuel model subs - FM40.csv")
table = parse_txt(blob)

# read in the encoded value list as numeric
from_codes = to_numeric(ee.List(table.get("Original FM40_number")))
# read the list of values to remap to as numeric
to_codes = to_numeric(ee.List(table.get("Updated FM40_number")))

print(from_codes.getInfo())
print(to_codes.getInfo())

[91, 92, 93, 98, 99, 101, 102, 103, 104, 121, 122, 123, 141, 142, 143, 144, 145, 146, 147, 161, 162, 163, 165, 181, 182, 183, 184, 185, 186, 187, 188, 189, 201, 202]
[91, 92, 93, 98, 99, 101, 101, 101, 101, 121, 121, 121, 141, 141, 141, 141, 142, 141, 142, 161, 161, 161, 161, 181, 181, 181, 181, 181, 181, 181, 181, 181, 201, 201]


<!-- For 2.3M intensity, SC and CC regions, we have re-treatments in road grid pixels, and unique floating point numbers distinguish a pixel that was treated twice from pixels only treated once. (See the `SCandCC_*_2Mcustom.ipynb` notebooks)

For run1 (Year5): 
* crosswalk only pixel value 1 (areas only treated once, no re-treatment)

For run2 (Year10):
* crosswalk pixel value 1.5 (areas treated in YI1 (Year5) and re-treated in YI2 (Year10))
* crosswalk pixel value 2 (areas only treated in YI2 (Year10))

For run3 (Year20):
* crosswalk pixel value 3.5 (areas treated in YI3 (Year15) and re-treated in YI4 (Year20))
* crosswalk pixel value 4 (areas only treated in YI4 (Year20)) -->

## 2.M SC/CC Custom Workflow
In `TreatmentSimulator_2MCustom` notebooks we exported `ee_treatments` and `retreatment_only` images separately

In this Notebook we:
1) FM40 crosswalk each treatment image separately for the 4 GF runs 

*run1 (Year5 | pixel value=1), run2 (Year10 | pixel value=2) , run3 (Year15 | pixel value=3) and run4 (Year20 | pixel value=4)*


2) then merging the two updated FM40s together in each GFrun pair, taking the min in overlapping pixels (remaps only go down, see above `from_codes` and `to_codes` lists)

In [19]:
# dumb mode - it works but i'm ashamed

# run 1 (pctGroup25)
if 'WUI' in ee_treated_path: # when we ran ee_treatmetns for WUI, 1's and 4's shared same footprint and 4 burned over 1's (took the max)
    run1_ee_treatment = ee_treated.eq(4).selfMask()
else:
    run1_ee_treatment = ee_treated.eq(1).selfMask() # for RFFC and Fire runs

run1_ee_treatment_remap = baseline_fm40.updateMask(run1_ee_treatment).remap(from_codes,to_codes)
run1_ee_treatment_fm40 = run1_ee_treatment_remap.unmask(baseline_fm40).rename('new_fm40').set('pctGroup',25)

# we don't re-treat in Year5 (run1 | YI1 whatever you want to call it) 
run1_fm40_mosaic = run1_ee_treatment_fm40 

# run 2 (pctGroup50)
run2_ee_treatment = ee_treated.eq(2).selfMask()
run2_retreatment = retreated.eq(2).selfMask()

run2_ee_treatment_remap = baseline_fm40.updateMask(run2_ee_treatment).remap(from_codes,to_codes)
run2_ee_treatment_fm40 = run2_ee_treatment_remap.unmask(baseline_fm40).rename('new_fm40').set('pctGroup',50)

run2_retreatment_remap = baseline_fm40.updateMask(run2_retreatment).remap(from_codes,to_codes)
run2_retreatment_fm40 = run2_retreatment_remap.unmask(baseline_fm40).rename('new_fm40').set('pctGroup',50)

run2_fm40_mosaic = (ee.Image.cat(run2_ee_treatment_fm40,
                                 run2_retreatment_fm40)
                    .reduce(ee.Reducer.min())
                    .rename('new_fm40')
                    .set('pctGroup',50))

# run 3  (pctGroup75)
run3_ee_treatment = ee_treated.eq(3).selfMask()
run3_retreatment = retreated.eq(3).selfMask()

run3_ee_treatment_remap = baseline_fm40.updateMask(run3_ee_treatment).remap(from_codes,to_codes)
run3_ee_treatment_fm40 = run3_ee_treatment_remap.unmask(baseline_fm40).rename('new_fm40').set('pctGroup',75)

run3_retreatment_remap = baseline_fm40.updateMask(run3_retreatment).remap(from_codes,to_codes)
run3_retreatment_fm40 = run3_retreatment_remap.unmask(baseline_fm40).rename('new_fm40').set('pctGroup',75)

run3_fm40_mosaic = (ee.Image.cat(run3_ee_treatment_fm40,
                                 run3_retreatment_fm40)
                    .reduce(ee.Reducer.min())
                    .rename('new_fm40')
                    .set('pctGroup',75))

updated_fm40s = [run1_fm40_mosaic,
                 run2_fm40_mosaic,
                 run3_fm40_mosaic]

# run 4  (only for RFFC and Fire runs, we've already handled WUI in run1 above)
run4_ee_treatment = ee_treated.eq(4).selfMask()
run4_retreatment = retreated.eq(4).selfMask()

run4_ee_treatment_remap = baseline_fm40.updateMask(run4_ee_treatment).remap(from_codes,to_codes)
run4_ee_treatment_fm40 = run4_ee_treatment_remap.unmask(baseline_fm40).rename('new_fm40').set('pctGroup',100)

run4_retreatment_remap = baseline_fm40.updateMask(run4_retreatment).remap(from_codes,to_codes)
run4_retreatment_fm40 = run4_retreatment_remap.unmask(baseline_fm40).rename('new_fm40').set('pctGroup',100)

run4_fm40_mosaic = (ee.Image.cat(run4_ee_treatment_fm40,
                                 run4_retreatment_fm40)
                    .reduce(ee.Reducer.min())
                    .rename('new_fm40')
                    .set('pctGroup',100))

if 'WUI' in ee_treated_path:
    updated_fm40s = [run1_fm40_mosaic,
                     run2_fm40_mosaic,
                     run3_fm40_mosaic,
                     run1_fm40_mosaic]
else:
    updated_fm40s = [run1_fm40_mosaic,
                    run2_fm40_mosaic,
                    run3_fm40_mosaic,
                    run4_fm40_mosaic]

In [20]:
import geemap
Map = geemap.Map()

Map.addLayer(baseline_fm40,{'min':90,'max':205},'baseline fm40')

# run 1 output steps
Map.addLayer(run1_ee_treatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},' run1 ee_treatments')
Map.addLayer(run1_fm40_mosaic,{'min':90,'max':205,},'updated fm40 mosaic run1') 

# run 2 output steps
Map.addLayer(run2_ee_treatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run2 ee_treatments')
Map.addLayer(run2_retreatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run2 retreated')
Map.addLayer(run2_ee_treatment_fm40,{'min':90,'max':205,},'run2 ee_treatment fm40')
Map.addLayer(run2_retreatment_fm40,{'min':90,'max':205,},'run2 retreatment fm40') 
Map.addLayer(run2_fm40_mosaic,{'min':90,'max':205,},'run2 updated fm40 mosaic') 

# run 3 output steps
Map.addLayer(run3_ee_treatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run3 ee_treatments')
Map.addLayer(run3_retreatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run3 retreated')
Map.addLayer(run3_ee_treatment_fm40,{'min':90,'max':205,},'run3 ee_treatment fm40')
Map.addLayer(run3_retreatment_fm40,{'min':90,'max':205,},'run3 retreatment fm40') 
Map.addLayer(run3_fm40_mosaic,{'min':90,'max':205,},'run3 updated fm40 mosaic') 

# run 4 output steps
Map.addLayer(run4_ee_treatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run4 ee_treatments')
Map.addLayer(run4_retreatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run4 retreated')
Map.addLayer(run4_ee_treatment_fm40,{'min':90,'max':205,},'run4 ee_treatment fm40')
Map.addLayer(run4_retreatment_fm40,{'min':90,'max':205,},'run4 retreatment fm40') 
Map.addLayer(run4_fm40_mosaic,{'min':90,'max':205,},'run4 updated fm40 mosaic') 

# Map.centerObject(treated,11)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Export each updated FM40 (run1,run2,run3)

In [21]:
# Export each ee_treatment FM40 in list
# use California as output region
ca = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("NAME","California")).first().geometry()

# use RUNID string as sub-folder in output Gdrive folder
runID_drive = os.path.basename(os.path.dirname(ee_treated_path)) # to get RUNID string

for img in list(range(4)):
    output = ee.Image(updated_fm40s[img]).toByte()
    pctGroup = output.get('pctGroup').getInfo()
    desc = f"UpdateFM40_pctGroup{pctGroup}"
    output_path = os.path.dirname(ee_treated_path)+"/"+f"FM40_pctGroup{pctGroup}"
  
    ee_treatments.export_img(output,desc,output_path,ca,30,'EPSG:3310')
    
    ee_treatments.export_image_to_drive(image=output,
                                        description=desc+'-Drive',
                                        region=ca,
                                        scale=30,
                                        crs='EPSG:3310',
                                        folder=f"MAS_FM40_for_GridFire",
                                        fileNamePrefix=f"{runID_drive}_FM40_pctGroup{pctGroup}",
                                        maxPixels=1e12)
    

Export Started for projects/mas-gee/assets/treatment_scenarios/RunID27/FM40_pctGroup25
Export Started (Drive): RunID27_FM40_pctGroup25
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID27/FM40_pctGroup50
Export Started (Drive): RunID27_FM40_pctGroup50
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID27/FM40_pctGroup75
Export Started (Drive): RunID27_FM40_pctGroup75
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID27/FM40_pctGroup100
Export Started (Drive): RunID27_FM40_pctGroup100
